In [72]:
import numpy as np
import xarray as xr
import pygrib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import pandas as pd
from scipy.constants import convert_temperature
import pytz
from datetime import datetime
import time
from metpy import calc
from metpy.units import units
import matplotlib as mpl
mpl.rc("savefig", dpi=300)

In [16]:
def calc_vpd_p(t, r):
    a = 21.874
    b = 7.66
    vpsat = 611*np.exp((a*(t-273.16)/(t-b)))
    return vpsat*(1 - r)/100

def calc_vpd_n(t, r):
    a = 17.269
    b = 35.86
    vpsat = 611*np.exp((a*(t-273.16)/(t-b)))    
    return vpsat*(1 - r)/100

In [43]:
for yl in range(15):
    temp_year = pygrib.open('./alaska_data/temp_{0}.grib'.format((2002+yl))) 
    dew_year = pygrib.open('./alaska_data/2m_dewpoint_{0}.grib'.format((2002+yl)))
    n = temp_year.messages
    VPD = np.zeros((n,161,291))
    VPD = np.ma.masked_array(VPD)
    
    for i in range(1,n+1):
        temp = temp_year.message(i).values
        Temp = convert_temperature(temp, 'Kelvin', 'Celsius')
        dew = dew_year.message(i).values
        dew = convert_temperature(dew, 'Kelvin', 'Celsius')
        rh = np.array([calc.relative_humidity_from_dewpoint(units.Quantity(Temp, 'celsius'), units.Quantity(dew, 'celsius'))])[0,:]
        vpd = np.where(temp>=273.16, calc_vpd_p(temp,rh), calc_vpd_n(temp,rh))
        VPD[i-1,:,:] = np.ma.masked_where(temp.mask, vpd)
        
    VPD = xr.DataArray(VPD)
    VPD.name = 'vpd'  
    VPD.to_netcdf('./alaska_data/VPD/vpd_{0}.nc'.format((2002+yl)))
    temp_year.close()
    dew_year.close()

0.8655240535736084